In [288]:
import json
import numpy as np
from itertools import product
from collections import defaultdict
with open("sodoku_puzzles.json", "r") as f:
    puzzles = json.load(f)

In [289]:
box0_coords = [(0, i) for i in range(3)] + [(1, i) for i in range(3)] + [(2, i) for i in range(3)]
box_coords = [[(t[0]+i, t[1]+j) for t in box0_coords] for i, j in product(range(0,9,3), range(0,9,3))]


class Puzzle:
    def __init__(self, puzzle: list[list[int]]):
        self._puzzle = [row.copy() for row in puzzle]
        
    def _make_repr(self):
        t = [
            "╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗",
            "╟───┼───┼───╫───┼───┼───╫───┼───┼───╢",
            "╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣",
            "╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝"
            ]
        o=[t[0]]
        for i,r in enumerate(self._puzzle):
            s="║"
            for j,v in enumerate(r):
                s+=f" {v if v else ' '} "
                if j<8:s+="║"if j in[2,5]else"│"
            o.append(s+"║")
            if i<8:o.append(t[2]if i in[2,5]else t[1])
        o.append(t[3])
        return"\n".join(o)
        
    def __repr__(self):
        return self._make_repr()
        
    def __getitem__(self, index: tuple[int, int]):
        if isinstance(index, int):
            i, j = index%9, index//9
        else:
            i, j = index
        if isinstance(j, slice):
            return [self._puzzle[_][i] for _ in list(range(9))[j]]
        return self._puzzle[j][i]
    
    def __setitem__(self, index, value):
        if isinstance(index, int):
            i, j = index%9, index//9
        else:
            i, j = index
        self._puzzle[j][i] = value
    
    def row(self, i, j):
        return set(self[:, j]) - {0}
    
    def col(self, i, j):
        return set(self[i, :]) - {0}
    
    def box(self, i, j):
        c, r = 3*(i//3), 3*(j//3)
        s = set()
        for row in self[c:c+3, r:r+3]:
            s.update(row)
        return s - {0}
    
    def rows(self):
        for i in range(9):
            yield self._puzzle[i]
    
    def cols(self):
        for i in range(9):
            yield [self._puzzle[j][i] for j in range(9)]
            
    def boxes(self):
        for bc in box_coords:
            yield [self[i,j] for i, j in bc]
    
    def valid(self) -> bool:
        for bits in self.rows():
            for i in range(1,10):
                if bits.count(i) > 1:
                    return False
        for bits in self.cols():
            for i in range(1,10):
                if bits.count(i) > 1:
                    return False
        for bits in self.boxes():
            for i in range(1,10):
                if bits.count(i) > 1:
                    return False
        return True
    
    def get_posibilites(self):
        posibilities = defaultdict(set)
        for i in range(9):
            for j in range(9):
                if self[i,j] != 0:
                    continue
                s = set(range(1,10))
                s -= self.row(i, j)
                s -= self.col(i, j)
                s -= self.box(i, j)
                posibilities[(i,j)] = s
        return posibilities                

In [290]:
def backtrack(p):
    zeros = [i for i in range(81) if p[i]==0]
    guesses = [0]*len(zeros)
    pos = 0
    while pos < len(zeros):
        while guesses[pos] < 9:
            guesses[pos] += 1
            p[zeros[pos]] = guesses[pos]
            if p.valid():
                pos += 1
                break
        else:
            guesses[pos] = 0
            p[zeros[pos]] = guesses[pos]
            pos -= 1

In [291]:
from time import perf_counter

In [292]:
results = []
for puz in puzzles['RawSudoku']:
    p = Puzzle(puz)
    n_clues = len([i for i in range(81) if p[i] != 0])
    tic = perf_counter()
    backtrack(p)
    toc = perf_counter() - tic 
    results.append((n_clues, toc))

In [293]:
results

[(30, 0.09740995801985264),
 (30, 2.5592681248672307),
 (30, 0.13023437513038516),
 (30, 0.2811563750728965),
 (30, 0.3600884173065424),
 (31, 1.024451124947518),
 (31, 0.7635502913035452),
 (30, 0.6509804590605199),
 (31, 0.5436001662164927),
 (30, 0.02831229194998741),
 (31, 0.40931783290579915),
 (30, 0.7779790419153869),
 (30, 0.04999366728588939),
 (30, 0.1746843750588596),
 (30, 0.037166791036725044),
 (30, 0.1721420418471098),
 (30, 0.08209695806726813),
 (31, 0.6038671252317727),
 (30, 0.1191779999062419),
 (30, 0.1298966254107654),
 (30, 1.6405621659941971),
 (30, 1.1460931664332747),
 (30, 0.01690233312547207),
 (30, 0.07530441740527749),
 (30, 0.20657441718503833),
 (31, 0.15428195800632238),
 (30, 2.0665871668606997)]

In [279]:
backtrack(p)
p

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 5 │ 1 │ 9 ║ 3 │ 2 │ 4 ║ 6 │ 7 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 6 │ 3 ║ 9 │ 1 │ 7 ║ 2 │ 5 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 2 │ 7 ║ 5 │ 8 │ 6 ║ 1 │ 9 │ 3 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 1 │ 9 │ 8 ║ 7 │ 6 │ 5 ║ 4 │ 3 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 3 │ 5 ║ 1 │ 4 │ 9 ║ 8 │ 6 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 7 │ 4 ║ 8 │ 3 │ 2 ║ 9 │ 1 │ 5 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 3 │ 8 │ 2 ║ 6 │ 5 │ 1 ║ 7 │ 4 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 4 │ 1 ║ 2 │ 9 │ 3 ║ 5 │ 8 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 5 │ 6 ║ 4 │ 7 │ 8 ║ 3 │ 2 │ 1 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝

In [265]:
t = 10
while t < 20:
    t += 1
    if t == 12:
        break
else:
    print("else")

In [235]:
l = [0,0,0,3,3,4]

In [253]:
# 10 -> (1,1)
# 62 -> (8,6)
# 80 -> (8,8)

In [255]:
i = 62
i%9, i//9

(8, 6)

In [252]:
10%9

1

In [200]:
while True:
    changes = 0
    for k, v in p.get_posibilites().items():
        if len(v) == 1:
            print(k)
            changes += 1
            p[k] = v.pop()
    if changes == 0:
        break

(8, 7)
(7, 7)


In [201]:
p

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 9 │   ║ 8 │ 6 │ 5 ║ 2 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 5 ║   │ 1 │ 2 ║   │ 6 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║   │ 4 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │ 8 ║   │ 5 │ 6 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 8 ║   │   │   ║ 4 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 5 │   ║ 9 │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 8 │   ║   │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 4 │   ║ 1 │ 7 │   ║ 5 │ 8 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 7 ║ 2 │ 8 │ 3 ║   │ 9 │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝

In [206]:
def split3(row):
    return [row[:3], row[3:6], row[6:]]

In [207]:
split3(p[0,:])

[[0, 0, 0], [0, 0, 4], [0, 2, 0]]

In [208]:
s = p.get_posibilites()

In [209]:
s[0,8]

{1, 5, 6}

In [195]:
class Puz:
    def __init__(self, puzzle):
        p = []
        for row in puzzle:
            p.extend(row)
        self._puzzle = np.array(p)

In [197]:
p = Puz(puzzles['RawSudoku'][0])
p._puzzle

array([0, 9, 0, 8, 6, 5, 2, 0, 0, 0, 0, 5, 0, 1, 2, 0, 6, 8, 0, 0, 0, 0,
       0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 8, 0, 5, 6, 0, 0, 8, 0, 0, 0, 4, 0,
       0, 4, 5, 0, 9, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0,
       1, 7, 0, 5, 0, 0, 0, 0, 7, 2, 8, 3, 0, 9, 0])